In [ ]:
LabelledPath = " " #Please provide path to labelled dataset
UnabelledPath = " " #Please provide path to unlabelled dataset

In [ ]:
import numpy as np
import h5py
from pickle import dump

In [ ]:
f = h5py.File(LabelledPath , "r")
H = f["H_Est"][:]
Pos = f["Pos"][:]
f.close()

In [ ]:
f = h5py.File(UnabelledPath , "r")
X = f["H_Est"][:]
f.close()

In [ ]:
import numpy as np

from tensorflow.keras.models import Model,Sequential , load_model
from tensorflow.keras.layers import Dense , Dropout, Conv2D ,Flatten,AveragePooling2D,Input,Conv2DTranspose,ZeroPadding2D
from tensorflow.keras.optimizers import Adam

def autoencoder():
    model = Sequential()
    model.add(Conv2D(128,(5,5),input_shape=(56,924,5),activation = 'relu' , padding='same'))
    model.add(AveragePooling2D((1,4)))
    model.add(Conv2D(32,(3,3),activation = 'relu' , padding='same'))
    model.add(AveragePooling2D((1,4)))
    model.add(Conv2D(32,(3,3),activation = 'linear', padding='same' ))
    model.add(Conv2DTranspose(32,(3,3) , strides=(1,1)  , padding='same',activation = 'relu' ))
    model.add(Conv2DTranspose(128,(3,3) , strides=(1,4)  , padding='same',activation = 'relu' ))
    model.add(ZeroPadding2D( ((0, 0), (2, 1))))
    model.add(Conv2DTranspose(5,(3,3) , strides=(1,4)  , padding='same',activation = 'linear' ))
    model.compile(loss='mean_squared_error', optimizer = Adam(1e-3)) 
    return model

In [ ]:
ae = autoencoder()
ae.summary()

In [ ]:
A = np.array(range(len(X)))
np.random.shuffle(A)
Train = A[:25000]
Val = A[25000:]

In [ ]:
def data_gen(X,bs=1):
    for i in range(0,len(X),bs):
        end = min(len(X),i+bs)
        yield X[i:end],X[i:end]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience = 80, verbose=1)
checkpointer = ModelCheckpoint('AE.h5', verbose=1, save_best_only=True)

In [ ]:
for i in range(30):
    ae.fit(data_gen(X[Train]), epochs=1, steps_per_epoch=len(Train)  ,callbacks=[earlystopper, checkpointer], validation_data=data_gen(X[Val]) ,validation_steps = len(Val))

In [ ]:
ae = load_model("AE.h5")
encoder = Model(ae.input,ae.layers[-5].output)

In [ ]:
H2 = encoder.predict(H)

Creating a custom test set

In [ ]:
from sklearn.model_selection import train_test_split
H_Train, H_Test , Pos_Train , Pos_Test = train_test_split(H2,Pos,test_size=0.05, random_state=42)

In [ ]:
def CNN():
    model = Sequential()
    model.add(Conv2D(32,(3,3),activation = 'linear', padding='same',input_shape=(56,57,32) ))
    model.add(Conv2D(16,(3,3),activation = 'linear', padding='same'))
    model.add(Flatten())
    model.add(Dense(512 ,activation = 'relu'))
    model.add(Dense(256 ,activation = 'relu' ))
    model.add(Dense(128 ,activation = 'relu' ))
    model.add(Dense(3))
    model.compile(loss="mean_absolute_error", optimizer = Adam(1e-3)) 
    return model

In [ ]:
model = CNN()
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience = 80, verbose=1)
checkpointer = ModelCheckpoint('best_CNN.h5', verbose=1, save_best_only=True)
hist = model.fit(H,Pos , epochs=1000, validation_split=0.2  ,callbacks=[earlystopper, checkpointer],batch_size=32)
val_loss = hist.history['val_loss']

In [ ]:
def mse(a,b):
    return np.sqrt(np.sum((a-b)**2,axis=1))

In [ ]:
y2 = model.predict(H_Test)

In [ ]:
rmse = np.mean(mse(y2,Pos_Test))
print(rmse)